In [1]:
import argparse

from pyspark.sql import SparkSession
from pyspark import SparkContext
import pyspark.sql.functions as F
import pyspark.sql.types as T

In [2]:

def load_config(spark_context: SparkContext):
    spark_context._jsc.hadoopConfiguration().set("fs.s3a.access.key", "minio_access_key")
    spark_context._jsc.hadoopConfiguration().set("fs.s3a.secret.key","minio_secret_key")
    spark_context._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "http://minio:9000")
    # spark_context._jsc.hadoopConfiguration().set("fs.s3a.access.key", os.getenv("AWS_ACCESS_KEY_ID", "<Your-MinIO-AccessKey>"))
    # spark_context._jsc.hadoopConfiguration().set("fs.s3a.secret.key",os.getenv("AWS_SECRET_ACCESS_KEY", "<Your-MinIO-SecretKey>"))
    # spark_context._jsc.hadoopConfiguration().set("fs.s3a.endpoint", os.getenv("ENDPOINT", "<Your-MinIO-Endpoint>"))
    spark_context._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "false")
    spark_context._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")
    # spark_context._jsc.hadoopConfiguration().set("fs.s3a.attempts.maximum", "1")
    # spark_context._jsc.hadoopConfiguration().set("fs.s3a.connection.establish.timeout", "5000")
    # spark_context._jsc.hadoopConfiguration().set("fs.s3a.connection.timeout", "10000")


spark = SparkSession.builder.getOrCreate()

load_config(spark.sparkContext)


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Reload4jLoggerFactory]
24/02/10 20:03:06 INFO SparkContext: Running Spark version 3.3.0-amzn-1
24/02/10 20:03:06 INFO ResourceUtils: ==============================================================
24/02/10 20:03:06 INFO ResourceUtils: No custom resources configured for spark.driver.

In [3]:
read_filepath = "s3a://gh-archive-data-raw/gh-archives/year=2023/month=01/day=01/hour=5/"

In [4]:
if __name__ == "__main__":
    # parser = argparse.ArgumentParser()
    # parser.add_argument("--date", help="Date in format YYYY-MM-DD", required=True)
    # parser.add_argument("--source_files_pattern", help="Source files pattern for the GH archive to process.", required=True)
    # parser.add_argument("--destination_files_pattern", help="Destination files pattern for the GH archive to process.", required=True)

    # args = parser.parse_args()
    # date = args.date

    # read_filepath = args.source_files_pattern
    # write_filepath = args.destination_files_pattern
    # read_filepath=read_filepath.format(date)
    # print(f"date received: {date}")


    print(f"read_filepath: {read_filepath}")
    df = spark.read.json(read_filepath)


    allowed_events = [
        "PushEvent",
        "ForkEvent",
        "PublicEvent",
        "WatchEvent",
        "PullRequestEvent",
    ]

    main_df = df.select(
        F.col("id").alias("event_id"),
        F.col("type").alias("event_type"),
        F.to_timestamp( F.col("created_at"), "yyyy-MM-dd'T'HH:mm:ss'Z'" ).alias("created_at"),
        F.col("repo.id").alias("repository_id"),
        F.col("repo.name").alias("repository_name"),
        F.col("repo.url").alias("repository_url"),
        F.col("actor.id").alias("user_id"),
        F.col("actor.login").alias("user_name"),
        F.col("actor.url").alias("user_url"),
        F.col("actor.avatar_url").alias("user_avatar_url"),
        F.col("org.id").alias("org_id"),
        F.col("org.login").alias("org_name"),
        F.col("org.url").alias("org_url"),
        F.col("org.avatar_url").alias("org_avatar_url"),
        F.col("payload.push_id").alias("push_id"),
        F.col("payload.distinct_size").alias("number_of_commits"),
        F.col("payload.pull_request.base.repo.language").alias("language"),
    ).filter(
        F.col("type").isin(allowed_events)
    )

    main_df = main_df.withColumn("year", F.year("created_at")) \
        .withColumn("month", F.month("created_at")) \
        .withColumn("day", F.dayofmonth("created_at")) \
        .withColumn("hour", F.hour("created_at")) \
        .withColumn("minute", F.minute("created_at")) \
        .withColumn("second", F.second("created_at")) \
    
    # add timestamp field
    main_df = main_df.withColumn("ts", F.unix_timestamp("created_at", "yyyy-MM-dd'T'HH:mm:ss'Z'"))


    # write the DataFrame to GCS partitioned by year, month, and day and bucketed by hour and minute
    # date = date.replace("-", "")

    # main_df.write \
    # .partitionBy("year", "month", "day") \
    # .bucketBy(24, "hour") \
    # .sortBy("hour", "minute") \
    # .option("path", write_filepath) \
    # .option("header", True) \
    # .mode("append") \
    # .saveAsTable(f"table{date}")

read_filepath: s3a://gh-archive-data-raw/gh-archives/year=2023/month=01/day=01/hour=5/


24/02/10 20:03:09 INFO SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
24/02/10 20:03:09 INFO SharedState: Warehouse path is 'file:/home/glue_user/workspace/jupyter_workspace/parquet/spark-warehouse'.
24/02/10 20:03:10 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
24/02/10 20:03:10 INFO MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
24/02/10 20:03:10 INFO MetricsSystemImpl: s3a-file-system metrics system started
24/02/10 20:03:11 WARN ApacheUtils: NoSuchMethodException was thrown when disabling normalizeUri. This indicates you are using an old version (< 4.5.8) of Apache http client. It is recommended to use http client version >= 4.5.9 to avoid the breaking change introduced in apache client 4.5.7 and the latency in exception handling. See https://github.com/aws/aws-sdk-java/issues/1919 for more information
24/02/10 20:03:11 INFO InMe

In [5]:
main_df.show()

24/02/10 20:03:23 INFO FileSourceStrategy: Pushed Filters: In(type, [ForkEvent,PublicEvent,PullRequestEvent,PushEvent,WatchEvent])
24/02/10 20:03:23 INFO FileSourceStrategy: Post-Scan Filters: type#15 IN (PushEvent,ForkEvent,PublicEvent,WatchEvent,PullRequestEvent)
24/02/10 20:03:23 INFO FileSourceStrategy: Output Data Schema: struct<actor: struct<avatar_url: string, display_login: string, gravatar_id: string, id: bigint, login: string ... 1 more field>, created_at: string, id: string, org: struct<avatar_url: string, gravatar_id: string, id: bigint, login: string, url: string ... 3 more fields>, payload: struct<action: string, before: string, comment: struct<_links: struct<html: struct<href: string>, pull_request: struct<href: string>, self: struct<href: string> ... 1 more fields>, author_association: string, body: string, commit_id: string, created_at: string ... 26 more fields>, commits: array<struct<author:struct<email:string,name:string>,distinct:boolean,message:string,sha:string,u

+-----------+----------------+-------------------+-------------+--------------------+--------------------+---------+-------------------+--------------------+--------------------+--------+----------+--------------------+--------------------+-----------+-----------------+----------+----+-----+---+----+------+------+----------+
|   event_id|      event_type|         created_at|repository_id|     repository_name|      repository_url|  user_id|          user_name|            user_url|     user_avatar_url|  org_id|  org_name|             org_url|      org_avatar_url|    push_id|number_of_commits|  language|year|month|day|hour|minute|second|        ts|
+-----------+----------------+-------------------+-------------+--------------------+--------------------+---------+-------------------+--------------------+--------------------+--------+----------+--------------------+--------------------+-----------+-----------------+----------+----+-----+---+----+------+------+----------+
|26164710177|      

24/02/10 20:03:24 INFO Executor: Finished task 0.0 in stage 1.0 (TID 1). 5805 bytes result sent to driver
24/02/10 20:03:24 INFO TaskSetManager: Finished task 0.0 in stage 1.0 (TID 1) in 807 ms on 931652ef741a (executor driver) (1/1)
24/02/10 20:03:24 INFO TaskSchedulerImpl: Removed TaskSet 1.0, whose tasks have all completed, from pool 
24/02/10 20:03:24 INFO DAGScheduler: ResultStage 1 (showString at NativeMethodAccessorImpl.java:0) finished in 0.869 s
24/02/10 20:03:24 INFO DAGScheduler: Job 1 is finished. Cancelling potential speculative or zombie tasks for this job
24/02/10 20:03:24 INFO TaskSchedulerImpl: Killing all running tasks in stage 1: Stage finished
24/02/10 20:03:24 INFO DAGScheduler: Job 1 finished: showString at NativeMethodAccessorImpl.java:0, took 0.874669 s
24/02/10 20:03:24 INFO CodeGenerator: Code generated in 23.23986 ms             


In [7]:
# write the DataFrame to Storage

main_df.write.format("parquet"). \
    mode("overwrite"). \
    save("s3a://gh-archive-data-curated/parquet/gh_archive/year=2023/month=01/day=01/hour=5")

24/02/10 20:05:37 INFO FileSourceStrategy: Pushed Filters: In(type, [ForkEvent,PublicEvent,PullRequestEvent,PushEvent,WatchEvent])
24/02/10 20:05:37 INFO FileSourceStrategy: Post-Scan Filters: type#15 IN (PushEvent,ForkEvent,PublicEvent,WatchEvent,PullRequestEvent)
24/02/10 20:05:37 INFO FileSourceStrategy: Output Data Schema: struct<actor: struct<avatar_url: string, display_login: string, gravatar_id: string, id: bigint, login: string ... 1 more field>, created_at: string, id: string, org: struct<avatar_url: string, gravatar_id: string, id: bigint, login: string, url: string ... 3 more fields>, payload: struct<action: string, before: string, comment: struct<_links: struct<html: struct<href: string>, pull_request: struct<href: string>, self: struct<href: string> ... 1 more fields>, author_association: string, body: string, commit_id: string, created_at: string ... 26 more fields>, commits: array<struct<author:struct<email:string,name:string>,distinct:boolean,message:string,sha:string,u

24/02/10 20:11:23 WARN NettyRpcEnv: Ignored failure: java.util.concurrent.TimeoutException: Cannot receive any reply from 931652ef741a:34407 in 10000 milliseconds
